# Glimpse of Transformer
## background

### zoo of transformer and their timeline

<img src="./imgs/transformer_timeline.png" width="400"/>

## core concepts

RNN(Recurrent Neural Network)recurrent architectures

> <a href="https://karpathy.github.io/2015/05/21/rnn-effectiveness/">The Unreasonable Effectiveness of Recurrent Neural Networks</a> by Andrej Karpathy

基本思想：通过隐藏状态（hidden state）存储历史信息，并在每个时间步更新。 适用于处理短期依赖任务，如小规模时间序列预测或短文本分类。

公式：

$ Y_h = tanh(W_hh_t-1 + W_xx_t + b) $

在transformer之前，早期NLP比较著名的RNN架构代表 LSTMs(Long Short-Term Memory) 。给模型一些输入（词或者字符），经过神经网络层之后输出一些vector，这些vector称之为 `hidden state`。
执行下一步之前会带入上一步的中间结果一并作为输入，用于做输出预测。广泛用于NLP任务，如语音处理、时间序列任务(库存预测、IOT传感器数据异常检测等）、机器翻译。

如下图: <br><br>

<img src="./imgs/RNN_arch.png" width="400"/>


### transformer core blocks

#### The encoder-decoder framework

通常来说 encoder decoder模块可以是任何能够对序列进行建模的神经网络架构。encoder将输入序列转换为对应序列向量，即`hidden_state` 或上下文；decoder主要基于encoder的隐藏状态迭代生成token，一次一个。<br><br>
 
<img src="./imgs/encoder_decoder_arch.png" width="400"/> 

在上述架构中，encoder模块最后一个隐藏状态（hidden state）会成为信息瓶颈。它必须表示整个输入序列的含义，因为在生成输出时，解码器只能访问这一信息。这对长序列来说尤其具有挑战性，因为在将所有信息压缩到单一固定表示的过程中，序列开头的信息可能会丢失。

#### Attention mechanisms

允许decoder模块访问所有encoder模块的隐藏状态，而不是由encoder模块最后一个隐藏状态节点承载所有整个输入序列的含义。从而解决上述的瓶颈。但也会导致输入到decoder的信息过多，故而引入注意力机制：即允许decoder在每个timestep对encoder每个状态设置不同权重。
`timestep` 代表模型处理序列中的某个位置的时间步

```
example: 句子 "I love machine learning" 被分解为 ["I", "love", "machine"
timestep=1 处理 "I"
timestep=2 处理 "love"
timestep=3 处理 "machine"
timestep=4 处理 "learning"
```
<br>

<img src="./imgs/attention_mechanism.png" width="400"/>

从上图可以看出，采用循环网络模型作为encoder decoder，处理序列都是串行的，性能是短板。因此出现了 `self-attention`，完全舍弃了循环机制，仍然保留允许访问同一网络层上所有状态的机制，且encoder decoder都有各自的 自注意力机制。

并将其输出传给前反馈神经网络做下一步处理。架构如下：<br/><br/>

<img src="./imgs/encoder_decoder_self_attention.png" width="400"/> <br>

<img src="./imgs/self_attention_with_contextual_emb.png" width="400"/>

#### Transfer learning

主要用于解决不具备条件从头训练一个模型，比如足够的或者满足条件的标注数据。通过迁移学习的思路，将模型分成head和body两部分，head部分是一个特定任务类型的网络，而body则是一个基于更泛化的原始模型网络，将其权重作为新预测任务模型的初始权重，
这样即可用较少的标注数据训练出高质量的新模型。<br/><br/>

<img src="./imgs/comparison_supervised_l_transfer_l.png" width="400" />

在计算机视觉领域，模型首先在诸如 ImageNet 这样的大规模数据集上进行训练，其中包含数百万张图片。这个过程称为预训练，其主要目的是让模型学习图像的基本特征，如边缘或颜色。然后，这些预训练模型可以在下游任务上进行微调，例如使用相对较少的标注样本（通常每个类别几百张）来分类花卉种类。微调后的模型通常比从零开始、仅使用相同数量标注数据进行监督训练的模型具有更高的准确率。

在 2017 年和 2018 年，多个研究团队提出了新的方法，最终让迁移学习在 NLP 领域得以实现。最早的突破来自 OpenAI 的研究人员，他们通过无监督预训练提取的特征，在情感分类任务上取得了优异的表现。随后，ULMFiT（Universal Language Model Fine-tuning for Text Classification 被提出，它引入了一种通用框架，使得预训练的 LSTM 模型能够适应各种 NLP 任务。

**ULMFiT 方法包含以下 三个关键步骤**

1. 语言模型预训练（LM Pretraining）

先在大规模无监督语料（如 Wikipedia）上训练一个语言模型（LSTM）。目标是预测下一个单词，学习语言结构和语义。

2. 目标任务的语言模型微调（LM Fine-tuning）

在目标任务的文本数据（较小规模）上继续训练语言模型，使其适应特定领域的语言风格和分布。

3. 任务特定的分类器微调（Task-specific Fine-tuning）

在目标任务（如情感分类、主题分类）上添加分类层，并使用分层微调（Discriminative Fine-tuning） 和 逐步解冻（Slanted Triangular Learning Rates, STLR） 技术，逐步适应新的任务，防止灾难性遗忘（Catastrophic Forgetting）。

**基于自注意力机制、迁移学习的模型对比** 

1. GPTGenerative Pre-trained Transformer）, 仅使用 Transformer 架构的解码器部分，并采用与 ULMFiT 相同的**语言建模（Language Modeling）** 方法。
    
    **关键特性：**
    
    * 仅使用 Transformer 的解码器（Decoder-only）。采用 自回归（Autoregressive） 方式，即通过预测下一个单词来训练模型（与 ULMFiT 语言建模类似）。由于解码器架构具有因果注意力（Causal Attention），每个单词只能关注其之前的单词，保证了生成任务的合理性。
    
    * 无监督预训练 + 任务微调。无监督预训练：在大量文本数据（如 BookCorpus）上训练语言模型，使其学习通用的语法和语义。微调（Fine-tuning）：在具体的 NLP 任务（如文本分类、问答、对话生成）上进行有监督训练，以适应不同应用场景。

2. BERT（Bidirectional Encoder Representations from Transformers） 仅使用 Transformer 架构的编码器部分（Encoder-only），并采用一种特殊的语言建模方法，称为 掩码语言建模（Masked Language Modeling, MLM）。

    **关键特性：**
    
    * Encoder-only。采用 Transformer 编码器（Encoder） 进行预训练，由于编码器可以同时关注整个句子的上下文，BERT 具备**双向（Bidirectional）** 的特性，使其能够学习更丰富的语言表示。
    
    * 掩码语言建模（Masked Language Modeling, MLM）。训练时，BERT 随机屏蔽（mask） 输入文本中的部分单词，并让模型预测被屏蔽的单词。
    ```
    例如，给定句子：
    "I looked at my [MASK] and saw that [MASK] was late."
    BERT 需要预测 [MASK] 处可能的单词，例如 "watch" 和 "train"。
    ```
    不同于 GPT 的 自回归（Autoregressive） 语言建模，它能让 BERT 同时考虑前后文信息，提高对上下文的理解能力。
> **自回归**（Autoregressive，简称 AR）是一种基于自身历史数据进行预测的建模方法，广泛应用于时间序列分析、语言建模和信号处理等领域。
> 在自然语言处理（NLP）中，自回归方法指的是基于已有的词或字符来预测下一个词或字符。例如，GPT（Generative Pre-trained Transformer）就是自回归语言模型的典型代表。

# Transformer anatomy
## Calculate attention weights(Scaled dot-production attention)
four steps: <br><br>
1. Project(映射) each token embedding into three vectors called query, key, and value
2. Compute attention scores.determine how much the query and key vectors relate to each other using a similarity function
3. Compute attention weights
4. Update the token embeddings.  attention weights multiply value vector<br><br>

    $ x'_i = \sum_j W_{ji} V{j} $
    <br><br>

*注意力权重计算 可视化示例：*

In [2]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"
show(model, "bert", tokenizer, text, display_mode="light",
layer=0, head=8)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

用图表示计算过程如下：<br><br>

<img src="./imgs/compute_scale_dot_production_process.png" width="400"/>

# Use Pytorch implement Transformer architecture

## tokenize the text

In [3]:
from transformers import AutoTokenizer

text = "time flies like an arrow"
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# add_special_tokens 是否生成特殊token [CLS] [SEP]
input = tokenizer(text, return_tensors="pt", add_special_tokens=False)
input.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])